In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import zipfile
import pickle
import time
import random
from dask import dataframe as dd

In [4]:
from IPython.display import clear_output

In [5]:
sys.path.append("../../alcohol replase/Scipts/")

In [6]:
import Utils as utl

In [7]:
device_position = ["Chest", "Hand", "Head", "Hip", "Forearm", "Shin", "Thigh", "UpperArm", "Waist"]
environment = ["Building", "Home", "Office", "Street", "Transportation"]
posture = ["Climbing (down)", "Climbing (up)", "Jumping", "Laydown", "Running", "Sitting", "Standing", "Walking"]
activity = ["DeskWork", "Eating/Drinking", "Housework", "Mealpreparation", "Movement", "PersonalGrooming", "Relaxing", 
            "Shopping", "Socializing", "Sleeping", "Sport", "Transportation"]
sub_activity = ["Somethingelse", "GotoWork", "WatchingTV", "TidyingUp", "GoHome", "atHome", "Breakfast", "Brunch", "CofeeBreak",
               "Dinner", "Lunch", "Snack", "Cleaning", "GoForAWalk", "Playing", "ListenToMusic", "Bar/Disco", "CinemaAtHome",
               "Party", "Basketball", "Bicycling", "Dancing", "Gym", "Gymnastics", "IceHockey", "Jogging", "Soccer", 
               "Bicycle", "Bus", "Car", "Motorcycle", "Scooter", "Skateboard", "Train", "Tram"]

In [8]:
device_position_lower = [str.lower(p) for p in device_position]
environment_lower = [str.lower(p) for p in environment]
posture_lower = [str.lower(p) for p in posture]
activity_lower = [str.lower(p) for p in activity]
sub_activity_lower = [str.lower(p) for p in sub_activity]

In [9]:
# the date time format for the dataset
date_time_format = "%d.%m.%y %H:%M:%S"
date_time_milli_seconds_format = "%d.%m.%y %H:%M:%S.%f"

acceleration_frequency = 50
orientation_frequency = 50

sensor_time_position = 1
sensor_value_position = [2, 3, 4]
sensor_label_environment_position = 5
sensor_label_posture_position = 6
sensor_label_device_position = 7
sensor_label_activity_position = 8

sensor_df_columns = ['id', 'attr_time', 'attr_x', 'attr_y', 'attr_z', 'label_environment',
       'label_posture', 'label_deviceposition', 'label_activity',
       'label_valid']

label_day_position = 1
label_subject_id_position = 2
label_start_time_position = 3
label_end_time_position = 4
label_position = 5

data_folder = "../dailylog2016_dataset/data/"

In [20]:
activity_class_dictionary  = {
    'Desk Work': 0,
    'Eating/Drinking': 1,
    'Housework': 2,
    'Meal preparation': 3,
    'Movement': 4,
    'Personal Grooming': 5,
    'Shopping': 6,
    'Socializing': 7,
    'Transportation': 8,
    'Sport': 9,
    'Relaxing': 10,
    'Take Medication': 11,
    'Sleeping': 12
}

posture_class_dictionary = {
    "Climbing (down)": 0,
    "Climbing (up)": 1,
    "Running": 2,
    "Sitting": 3,
    "Standing": 4,
    "Walking": 5
}

activity_file_path = "../Processed data/adl_activity_dataset.pickle"
posture_file_path = "../Processed data/adl_posture_dataset.pickle"
posture_full_file_path = "../Processed data/adl_posture_dataset_full.pickle"

- Acceleration (50 Hz), Orientation (50 Hz), GPS (every 10 minutes)
- Labels: Activity, Device Position, Environment, Posture




- Device Posistion: Chest, Hand, Head, Hip, Forearm, Shin, Thigh, Upper Arm, Waist
- Environment: Building, Home, Office, Street, Transportation
- Posture: Climbing, Jumping, Lay, Running, Sitting, Standing, Walking







- Activity - Subactivity
- Desk Work - N/A
- Eating / Drinking - Breakfast, Brunch, Cofee Break, Dinner, Lunch Snack
- Housework - Cleaning, Tidying Up
- Meal Preparation - N/A
- Movement - Go for a Walk, Go Home, Go to Work
- Personal Grooming - N/A
- Relaxing - Playing, Listen to Music, Watching TV
- Shopping - N/A
- Socializing - Bar / Disco, Cinema at Home
- Sleeping - N/A
- Sport - Basketball, Bicycling, Dancing, Gym, Gymnastics, Ice Hockey, Jogging, Soccer
- Transportation - Bicycle, Bus, Car, Motorcycle, Scooter, Skateboard, Train, Tram



In [10]:
# sliding window of size 2.56 seconds with 50% overlap i.e., 128 values at 50Hz sampling frequency
window_duration = 2.56
overlap_percent = 0.5
window_length = int(window_duration * acceleration_frequency)
n_channels = 3

In [11]:
def get_class_data(total_data, class_):
    if len(total_data) < window_length:
        return []
    
    # get the class data
    index = np.where(total_data[:, -1] == class_)[0]
    class_data = total_data[index, :3]
    
    # get the X, Y, and Z axis values
    class_data_x = class_data[:, 0]
    class_data_y = class_data[:, 1]
    class_data_z = class_data[:, 2]
    
    
    # now run the windowing methods on each of the axis data
    class_segments_x = utl.segment_sensor_reading(class_data_x, window_duration, overlap_percent, acceleration_frequency)
    class_segments_y = utl.segment_sensor_reading(class_data_y, window_duration, overlap_percent, acceleration_frequency)
    class_segments_z = utl.segment_sensor_reading(class_data_z, window_duration, overlap_percent, acceleration_frequency)
    
#     print(class_segments_x.shape)
#     print(class_segments_y.shape)
#     print(class_segments_z.shape)

    class_data = np.concatenate([class_segments_x, class_segments_y], axis = 1)
    class_data = np.concatenate([class_data, class_segments_z], axis =  1)
    class_data = class_data.reshape(-1, window_length, n_channels)
    
    print("Class segments shape {}".format(class_data.shape))
    return class_data

In [30]:
def select_random_samples(x, y, n_samples):
    length = len(x)
    if length == n_samples:
        return x, y
    
    random_index = np.random.randint(0, length, n_samples)
    return x[random_index], y[random_index]

In [31]:
def balance_dataset(x, y):
    classes, counts = np.unique(y, return_counts=True)
    min_value = np.min(counts)
    
    X_ = np.empty((1, window_length, n_channels))
    Y_ = np.empty(1)
    
    for cls in classes:
        index = np.where(y == cls)[0]
        x_tp = x[index]
        y_tp = y[index]
        
        x_tp, y_tp = select_random_samples(x_tp, y_tp, min_value)
        X_ = np.concatenate([X_, x_tp])
        Y_ = np.concatenate([Y_, y_tp])
        
    return X_[1:, ], Y_[1:]

In [22]:
def get_x_y_from_dict(data_dict, label_dict):
    X = np.empty((1, window_length, n_channels))
    Y = np.empty(1)

    for label in label_dict.keys():
        segments = np.array(data_dict[label])
        print(label, segments.shape)
        X = np.concatenate([X, segments])
        Y = np.concatenate([Y, np.ones(segments.shape[0], dtype='int') * label_dict[label]])
    
    return X[1:, ], Y[1:]

In [14]:
subject_indexes = np.array([1, 2, 3, 4, 5, 6, 7])

In [15]:
source_ids = subject_indexes[random.sample(range(0, max(subject_indexes)), len(subject_indexes)//2)]
target_ids = [r for r in subject_indexes if r not in source_ids]

In [16]:
source_ids, target_ids

(array([1, 7, 6]), [2, 3, 4, 5])

In [17]:
def extract_subject_folders(data_folder, label_type, subject_ids=[]):
    # container to store the processed data; we have 4 values in each row
    processed_data_dict = {}
    
    # for each subject id
    for subject in subject_ids:
        # for each subject folder, we have data and image folder
        subject_data_folder = data_folder + "subject" + str(subject) + "/" + "data/"
            
        # now for each extracted acc csv file
        try:
            csv_files = os.listdir(subject_data_folder + "acc_csv/")
        except:
            continue
            
        for csv_file in csv_files:
            clear_output(wait=True)
            file_path = subject_data_folder + "acc_csv/" + csv_file
            print("Processing file : " + file_path)
            
            # load the CSV file
            data_df = pd.read_csv(file_path).to_numpy()#dd.read_csv(file_path).values #
            
            # get the unique labels in the data
            unique_labels = np.unique(data_df[:, label_type])
            unique_labels = np.delete(unique_labels, np.where(unique_labels == 'unknown'))
            
            # for each unique label
            for label in unique_labels:
                # get the index for this label
                indexes = np.where(data_df[:, label_type] == label)[0]

                # get the data at the indexes
                data_for_label = data_df[indexes]
                # get the sensor value 2:5 and the label for the specified type
                data_for_label = data_for_label[:, np.r_[2:5, label_type]]
                
                print(label, data_for_label.shape)
                
                # convert the sensor values into windows
                data_for_label = get_class_data(data_for_label, label)
                try:
                    processed_data_dict[label].extend(data_for_label)
                except:
                    processed_data_dict[label] = []
                    processed_data_dict[label].extend(data_for_label)
                    
    return processed_data_dict

In [18]:
source_dataset = extract_subject_folders(data_folder, sensor_label_posture_position, source_ids)

Processing file : ../dailylog2016_dataset/data/subject6/data/acc_csv/SensorAccelerometerData_labeled_day9.csv
Climbing (down) (12706, 4)
Class segments shape (197, 128, 3)
Climbing (up) (3846, 4)
Class segments shape (59, 128, 3)
Recumbency (332796, 4)
Class segments shape (5198, 128, 3)
Sitting (1121240, 4)
Class segments shape (17518, 128, 3)
Standing (398988, 4)
Class segments shape (6233, 128, 3)
Walking (56987, 4)
Class segments shape (889, 128, 3)


In [32]:
source_x, source_y = get_x_y_from_dict(source_dataset, posture_class_dictionary)

Climbing (down) (2144, 128, 3)
Climbing (up) (1653, 128, 3)
Running (8520, 128, 3)
Sitting (525000, 128, 3)
Standing (148572, 128, 3)
Walking (58461, 128, 3)


In [33]:
source_x, source_y = balance_dataset(source_x, source_y)

In [36]:
source_y = np.array(source_y, dtype=int)

In [38]:
f = open("../Processed data/DL_source_dataset.pickle", "wb")
pickle.dump([source_x, source_y, source_ids], f)
f.close()

In [39]:
target_dataset = extract_subject_folders(data_folder, sensor_label_posture_position, target_ids)

Processing file : ../dailylog2016_dataset/data/subject4/data/acc_csv/SensorAccelerometerData_labeled_day9.csv
Nicht festgelegt (79058, 4)
Class segments shape (1234, 128, 3)
gehen (592486, 4)
Class segments shape (9256, 128, 3)
liegen (1066407, 4)
Class segments shape (16661, 128, 3)
sitzen (4148886, 4)
Class segments shape (64825, 128, 3)
springen (106, 4)
stehen (1033418, 4)
Class segments shape (16146, 128, 3)


In [40]:
target_x, target_y = get_x_y_from_dict(target_dataset, posture_class_dictionary)

Climbing (down) (1132, 128, 3)
Climbing (up) (1086, 128, 3)
Running (1859, 128, 3)
Sitting (338832, 128, 3)
Standing (99160, 128, 3)
Walking (22973, 128, 3)


In [41]:
target_x, target_y = balance_dataset(target_x, target_y)

In [42]:
target_y = np.array(target_y, dtype=int)

In [43]:
f = open("../Processed data/DL_target_dataset.pickle", "wb")
pickle.dump([target_x, target_y, target_ids], f)
f.close()

In [14]:
'''
    Extract data from the accelerometer files for the given label type. 
    @param: data_folder, path to the folder with subjects folder
    @param: label_type, which label type to extract from the accelerometer data files. Possible values are 
        sensor_label_environment_position = 5 for environment labels
        sensor_label_posture_position = 6 for posture labels
        sensor_label_device_position = 7 for device position labels
        sensor_label_activity_position = 8 for activity labels
        
    @param: open_zips, whether to open the acc_csv zip files or not before extracting the sensor values for the given label type
    
'''
def combine_subject_files(data_folder, label_type, open_zips = False):
    # container to store the processed data; we have 4 values in each row
    processed_data_dict = {}
    
    # get the subject folders
    subject_folders = os.listdir(data_folder)
    
    # for each subject folder
    for folder in subject_folders:
        # for each subject folder, we have data and image folder
        subject_data_folder = data_folder + folder + "/" + "data/"
        
        # now at this location we have acc_csv zip file. 
        if open_zips == True:
            #open acc_csv zip to get the 14 days data for each subject
            zip_file = subject_data_folder + "acc_csv.zip"
            try:
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall(subject_data_folder + "acc_csv/")
            except:
                # if not able to open the zip file then just continue to next subject
                continue
            
        # now for each extracted acc csv file
        try:
            csv_files = os.listdir(subject_data_folder + "acc_csv/")
        except:
            continue
            
        for csv_file in csv_files:
            clear_output(wait=True)
            file_path = subject_data_folder + "acc_csv/" + csv_file
            print("Processing file : " + file_path)
            
            # load the CSV file
            data_df = pd.read_csv(file_path).to_numpy()#dd.read_csv(file_path).values #
            
            # get the unique labels in the data
            unique_labels = np.unique(data_df[:, label_type])
            unique_labels = np.delete(unique_labels, np.where(unique_labels == 'unknown'))
            
            # for each unique label
            for label in unique_labels:
                # get the index for this label
                indexes = np.where(data_df[:, label_type] == label)[0]

                # get the data at the indexes
                data_for_label = data_df[indexes]
                # get the sensor value 2:5 and the label for the specified type
                data_for_label = data_for_label[:, np.r_[2:5, label_type]]
                
                print(label, data_for_label.shape)
                
                # convert the sensor values into windows
                data_for_label = get_class_data(data_for_label, label)
                try:
                    processed_data_dict[label].extend(data_for_label)
                except:
                    processed_data_dict[label] = []
                    processed_data_dict[label].extend(data_for_label)
                    
    return processed_data_dict

In [14]:
start = time.time()
dataset = combine_subject_files(data_folder, sensor_label_posture_position)
end = time.time()
print("Read csv time: ",(end-start),"sec")

Processing file : ../dailylog2016_dataset/data/subject7/data/acc_csv/SensorAccelerometerData_labeled_day9.csv
Nicht festgelegt (609428, 4)
Class segments shape (9521, 3, 128)
gehen (266100, 4)
Class segments shape (4156, 3, 128)
klettern/steigen (hoch) (3405, 4)
Class segments shape (52, 3, 128)
klettern/steigen (runter) (3458, 4)
Class segments shape (53, 3, 128)
sitzen (1040180, 4)
Class segments shape (16251, 3, 128)
stehen (19546, 4)
Class segments shape (304, 3, 128)
Read csv time:  580.0130429267883 sec


In [15]:
for key in dataset.keys():
    print(key, np.array(dataset[key]).shape)

Climbing (down) (3276, 3, 128)
Climbing (up) (2739, 3, 128)
Not Specified (98511, 3, 128)
Sitting (863832, 3, 128)
Standing (247732, 3, 128)
Walking (81434, 3, 128)
Recumbency (65660, 3, 128)
Running (10379, 3, 128)
Jumping (4, 3, 128)
Nicht festgelegt (80584, 3, 128)
gehen (146026, 3, 128)
klettern/steigen (hoch) (2110, 3, 128)
klettern/steigen (runter) (2857, 3, 128)
rennen (7613, 3, 128)
sitzen (1023402, 3, 128)
stehen (173008, 3, 128)
springen (1477, 3, 128)
liegen (25977, 3, 128)


In [34]:
X, Y = get_x_y_from_dict(dataset, posture_class_dictionary)

Climbing (down) (3276, 3, 128)
Climbing (up) (2739, 3, 128)
Running (10379, 3, 128)
Sitting (863832, 3, 128)
Standing (247732, 3, 128)
Walking (81434, 3, 128)


In [36]:
X.shape, Y.shape

((1209392, 3, 128), (1209392,))

In [39]:
file_path = posture_full_file_path
f = open(file_path, "wb")
pickle.dump([X, Y, posture_class_dictionary], f)
f.close()

MemoryError: 

In [40]:
whos

Variable                            Type              Data/Info
---------------------------------------------------------------
X                                   ndarray           1209392x3x128: 464406528 elems, type `object`, 3715252224 bytes (3543.140625 Mb)
Y                                   ndarray           1209392: 1209392 elems, type `float64`, 9675136 bytes (9.2269287109375 Mb)
absolute_import                     _Feature          _Feature((2, 5, 0, 'alpha<...>0, 0, 'alpha', 0), 16384)
acceleration_frequency              int               50
activity                            list              n=12
activity_class_dictionary           dict              n=13
activity_file_path                  str               ../Processed data/adl_activity_dataset.pickle
activity_lower                      list              n=12
clear_output                        function          <function clear_output at 0x00000224AD3901F8>
combine_subject_files               function          <function 